In [1]:
#libraries used 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [2]:
# URLs for the Red and White wine datasets
red_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
white_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

# Load the data - Note the sep=';'
red_wine = pd.read_csv(red_url, sep=';')
white_wine = pd.read_csv(white_url, sep=';')

# Professional Step: Add a column to distinguish them before merging
red_wine['type'] = 'red'
white_wine['type'] = 'white'

# Combine them into one "Master" dataframe
df = pd.concat([red_wine, white_wine])

print(f"Dataset loaded! Total rows: {len(df)}")
print(df.head())

Dataset loaded! Total rows: 6497
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality type  
0      9.4        5  red  

In [3]:
red_wine.shape

(1599, 13)

In [4]:
white_wine.shape

(4898, 13)

In [5]:
df_red = pd.DataFrame(red_wine)
df_red

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,red
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,red
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,red
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,red


In [6]:
df_white = pd.DataFrame(white_wine)
df_white

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,white
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,white
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,white
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


In [7]:
df_white.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
 12  type                  4898 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 497.6+ KB


In [8]:
#separateing 
# - Low:       0-4 (4 exclusive)
# - Medium Low: 4-5 ( exclusive)
# - Medium High: 6-7 (6 exclusive)
# - High:      6-7 (7 exclusive)
# - Very High: 7-10 (10 exclusive)

bins = [0, 4, 6, 7, 8, 10]
choices = ['Low', 'Medium Low', 'Medium High', 'High', 'Very High']

df_white['quality_category'] = pd.cut(df_white['quality'], bins=bins, labels=choices, right=False)

# Verify the distribution of the new quality categories
print(df_white['quality_category'].value_counts())

quality_category
Medium High    2198
Medium Low     1620
High            880
Very High       180
Low              20
Name: count, dtype: int64


In [9]:
#Converting categorical into numerical, dropping 'type' column 
df_white_cleaned = df_white.drop(columns = ['type']) 

#Initializing the encoder
le = LabelEncoder()

#Putting the encoded quality category into one column 
df_white_cleaned['quality_category_encoded'] = le.fit_transform(df_white_cleaned['quality_category'])

In [10]:
#Separating Feature columns from target one for white wine 
features = df_white_cleaned.drop(columns = ["quality_category_encoded", "quality_category"])
target = df_white_cleaned["quality_category_encoded"]
df_white_cleaned

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_category,quality_category_encoded
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,Medium High,2
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,Medium High,2
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,Medium High,2
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,Medium High,2
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,Medium High,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,Medium High,2
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,Medium Low,3
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,Medium High,2
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,High,0


In [11]:
#Train/Test Separation 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=42)

In [12]:
#MinMax Scaling & Transformation 0-1
normalizer = MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)
X_train_norm, X_test_norm

(array([[0.4375    , 0.08823529, 0.21686747, ..., 0.22093023, 0.70967742,
         0.5       ],
        [0.3125    , 0.16666667, 0.26506024, ..., 0.34883721, 0.24193548,
         0.33333333],
        [0.225     , 0.23529412, 0.19879518, ..., 0.31395349, 0.5       ,
         0.5       ],
        ...,
        [0.475     , 0.18627451, 0.31325301, ..., 0.36046512, 0.5483871 ,
         0.5       ],
        [0.3125    , 0.15686275, 0.1746988 , ..., 0.18604651, 0.41935484,
         0.5       ],
        [0.5375    , 0.18627451, 0.21084337, ..., 0.47674419, 0.38709677,
         0.83333333]], shape=(3918, 12)),
 array([[0.275     , 0.20588235, 0.24698795, ..., 0.43023256, 0.47849462,
         0.66666667],
        [0.2       , 0.44117647, 0.09638554, ..., 0.36046512, 0.83870968,
         0.83333333],
        [0.4125    , 0.16666667, 0.23493976, ..., 0.24418605, 0.67741935,
         0.83333333],
        ...,
        [0.2625    , 0.17647059, 0.12650602, ..., 0.24418605, 0.24193548,
         0.5    

In [40]:
#using KNN to train the model, checking for accuracy 
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_norm, y_train)
y_pred = knn.predict(X_test_norm)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.9469387755102041


In [13]:
#AdaBoosting 
ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=20),n_estimators=100)
ada_reg.fit(X_train_norm, y_train)
AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=20),n_estimators=100)
pred = ada_reg.predict(X_test_norm)
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", ada_reg.score(X_test_norm, y_test))

MAE 0.0
RMSE 0.0
R2 score 1.0


#Gradient Boosting 
gb_reg = GradientBoostingRegressor(max_depth=20,n_estimators=100)
gb_reg.fit(X_train_norm, y_train)
GradientBoostingRegressor(max_depth=20)
pred = gb_reg.predict(X_test_norm)
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test))
print("R2 score", gb_reg.score(X_test_norm, y_test))

In [14]:
#Random Forest 
forest = RandomForestRegressor(n_estimators=100,max_depth=20)
forest.fit(X_train_norm, y_train)
RandomForestRegressor(max_depth=20)
pred = forest.predict(X_test_norm)
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(y_test, pred))
print("R2 score", forest.score(X_test_norm, y_test))

MAE 0.0
RMSE 0.0
R2 score 1.0


In [15]:
#Bagging & Pasting 
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),n_estimators=100,max_samples = 1000)
bagging_reg.fit(X_train_norm, y_train)
BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),max_samples=1000, n_estimators=100)
pred = bagging_reg.predict(X_test_norm)
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(y_test, pred))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.0007142857142857149
RMSE 0.008005100414926808
R2 score 0.9999489011641375
